In [1]:
import zipfile
import os

# Définir le chemin du fichier ZIP et le répertoire de destination
zip_file_path = '/media/zf31/YOLOCSDtest.zip'
extract_to_path = '/media/DOLPHIN_ALEXIS/Analyses_alexis/YOLOCSDtest'

# Path to the video file
video_path = '/media/DOLPHIN_ALEXIS/2023/Exp_01_Jun_2024_1145_cam1-4.mp4'


# Décompresser le fichier ZIP
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_path)

# Lister les fichiers extraits
extracted_files = os.listdir(extract_to_path)
extracted_files

['obj_train_data', 'obj.data', 'obj.names', 'train.txt']

In [2]:
import os

labels_folder = os.path.join(extract_to_path, 'obj_train_data')


def remove_empty_files(path):
    for file in os.listdir(path):
        if file.endswith(".txt"):
            file_path = os.path.join(path, file)
            if os.path.isfile(file_path) and os.path.getsize(file_path) == 0:
                os.remove(file_path)

# Replace 'your_directory_path' with the path to your directory
remove_empty_files(labels_folder) # Remove empty label files

# Get the list of files in the directory
files = os.listdir(labels_folder)

# Check if the list is not empty
if files:
    # Sort the files in ascending order
    files.sort(reverse=False)

    # Get the label value from the last file in the directory
    label_value = int(files[-1].split('.')[0].split("_")[1].zfill(6))
    label_value
else:
    label_value = 0

label_value

15812

In [4]:
import cv2
import os

video_path = '/media/DOLPHIN_ALEXIS/2023/Exp_01_Jun_2024_1145_cam1-4.mp4'
# Directory to save the extracted frames
output_dir = os.path.join(extract_to_path, 'images')

# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Open the video file
video = cv2.VideoCapture(video_path)

# Read the first frame
success, frame = video.read()
frame_count = 0

# Loop through the video frames
while success and frame_count <= label_value:
    # Save the frame as a JPG image
    frame_path = os.path.join(output_dir, f'frame_{frame_count:06d}.jpg')
    cv2.imwrite(frame_path, frame)

    # Read the next frame
    success, frame = video.read()
    frame_count += 1

# Release the video file
video.release()

In [9]:
import yaml
# Define the classes
classes = ['dolphin', 'fish', 'diver']

# Define the data dictionary
data_yaml = {
    'train': output_dir,
    'val': output_dir,
    'nc': len(classes),
    'names': classes
}


yaml_path = os.path.join(extract_to_path,'data_yaml.yaml')
# Write the data dictionary to the data.yaml file
with open(yaml_path, 'w') as f:
    yaml.dump(data_yaml, f)

In [11]:
%cd /users/zfne/emanuell/Documents/GitHub/yolov5/

/export/home1/users/zfne/emanuell/Documents/GitHub/yolov5


In [12]:
# Train the model
!python train.py --img 640 --batch 16 --epochs 100 --data yaml_path --cfg yolov5s.yaml --weights yolov5s.pt

2024-06-12 14:02:49.226453: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-12 14:02:49.226492: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-12 14:02:49.227549: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s.pt, cfg=yolov5s.yaml, data=yaml_path, hyp=data/hyps/hyp.scratch-low.yaml, epochs=100, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls

In [4]:
# Evaluate the model
!python val.py --data /kaggle/working/data_yaml.yaml --weights runs/train/exp/weights/best.pt

[{'frame_id': 0,
  'label': 'diver',
  'x_center': 372.675,
  'y_center': 125.515,
  'width': 184.11,
  'height': 109.03},
 {'frame_id': 1,
  'label': 'diver',
  'x_center': 373.0,
  'y_center': 124.5,
  'width': 184.0,
  'height': 109.0},
 {'frame_id': 2,
  'label': 'diver',
  'x_center': 372.0,
  'y_center': 122.5,
  'width': 184.0,
  'height': 109.0},
 {'frame_id': 3,
  'label': 'diver',
  'x_center': 371.0,
  'y_center': 120.5,
  'width': 184.0,
  'height': 109.0},
 {'frame_id': 4,
  'label': 'diver',
  'x_center': 370.0,
  'y_center': 120.5,
  'width': 184.0,
  'height': 109.0}]

In [33]:
import cv2
import torch
from models.common import DetectMultiBackend
from utils.general import non_max_suppression

# Check if GPU is available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')

# Load YOLOv5 model
device = torch.device(device)
model = DetectMultiBackend('/users/zfne/emanuell/Documents/GitHub/yolov5/runs/train/exp3/weights/last.pt', device=device)
model.eval()

# Define a function to resize frames to the model's expected size
def resize_frame_to_model_size(frame, model_stride=32):
    height, width = frame.shape[:2]
    new_height, new_width = (height // model_stride) * model_stride, (width // model_stride) * model_stride
    resized_frame = cv2.resize(frame, (new_width, new_height))
    return resized_frame

# Define a function to process video frames
def process_frame(frame, model):
    # Resize the frame to ensure it is compatible with the model
    frame = resize_frame_to_model_size(frame)
    
    # Preprocess the frame for YOLOv5
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img = img / 255.0
    img = torch.from_numpy(img).float().to(device).permute(2, 0, 1).unsqueeze(0)

    # Run YOLOv5 on the frame
    with torch.no_grad():
        pred = model(img)

    # Apply NMS
    pred = non_max_suppression(pred)[0]
    
    # Extract bounding boxes and labels
    boxes = pred.cpu().numpy()
    labels = model.names
    
    return boxes, labels

# Define a function to annotate frames with detected bounding boxes
def annotate_frame(frame, boxes, labels):
    for box in boxes:
        x1, y1, x2, y2, conf, cls = box
        if labels[int(cls)] == 'diver' or labels[int(cls)] == 'dolphin' or labels[int(cls)] == 'fish':
            # Draw bounding box
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
            # Add label
            cv2.putText(frame, f'{labels[int(cls)]} {conf:.2f}', (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    
    return frame

# Load the video file
video_path = '/media/DOLPHIN_ALEXIS/2023/Exp_01_Jun_2024_1145_cam1-4.mp4'
cap = cv2.VideoCapture(video_path)

# Get video properties
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define the codec and create VideoWriter object
output_path = '/media/DOLPHIN_ALEXIS/output_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# Check if video loaded successfully
if not cap.isOpened():
    print("Error opening video stream or file")

# Process the video frame by frame
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    # Ensure frame is in the correct format
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the entire frame
    boxes, labels = process_frame(frame_rgb, model)

    # Annotate the frame with the detected boxes and labels
    annotated_frame = annotate_frame(frame, boxes, labels)

    # Write the annotated frame to the output video file
    out.write(annotated_frame)
    
# Release video capture and writer objects
cap.release()
out.release()

print("Video processing complete. The output video is saved to /media/DOLPHIN_ALEXIS/output_video.mp4")

Annotations YOLO créées avec succès.


Créer train et val 5%

In [37]:
import os
import random
import shutil

# Define the paths
train_images_dir = '/home/alexis/Desktop/New_caméra/Custom_dataset/train/images'
train_labels_dir = '/home/alexis/Desktop/New_caméra/Custom_dataset/train/labels'
val_images_dir = '/home/alexis/Desktop/New_caméra/Custom_dataset/val/images'
val_labels_dir = '/home/alexis/Desktop/New_caméra/Custom_dataset/val/labels'

# Create the val directories if they don't exist
os.makedirs(val_images_dir, exist_ok=True)
os.makedirs(val_labels_dir, exist_ok=True)

# Get the list of image files in the train directory
labels_files = os.listdir(train_labels_dir)

# Calculate the number of images to move
num_images_to_move = int(len(labels_files) * 0.05)

# Randomly select the images to move
labels_to_move = random.sample(labels_files, num_images_to_move)

# Move the selected images and labels to the val directory
for label_file in labels_to_move:
    # Move the image file
    image_file = label_file.replace('.txt', '.jpg')
    src_image_path = os.path.join(train_images_dir, image_file)
    dest_image_path = os.path.join(val_images_dir, image_file)
    shutil.move(src_image_path, dest_image_path)

    # Move the corresponding label file
    src_label_path = os.path.join(train_labels_dir, label_file)
    dest_label_path = os.path.join(val_labels_dir, label_file)
    shutil.move(src_label_path, dest_label_path)

# Print the number of images moved
print(f"{num_images_to_move} images and labels moved from train to val directory.")


134 images moved from train to val directory.


In [31]:
# Delete the images without labels
for image_file in os.listdir(train_images_dir):
    label_file = image_file.replace('.jpg', '.txt')
    if label_file not in labels_files:
        image_path = os.path.join(train_images_dir, image_file)
        os.remove(image_path)

In [38]:
import yaml

# Define the classes
classes = ['dolphin', 'diver', 'fish']

# Define the data dictionary
data = {
    'train': '/home/alexis/Desktop/New_caméra/Custom_dataset/train/images',
    'val': '/home/alexis/Desktop/New_caméra/Custom_dataset/val/images',
    'nc': len(classes),
    'names': classes
}

# Write the data dictionary to the data.yaml file
with open('/home/alexis/Desktop/New_caméra/Custom_dataset/data.yaml', 'w') as f:
    yaml.dump(data, f)
